In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
import numpy as np
import tensorflow as tf
from VGG16_GAP import VGG16_GAP
from VGG16_flatten import VGG16_flatten

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [3]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [4]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [5]:
with open('save/inv_label_dict.pkl', 'rb') as f:
    inv_y_dict = pickle.load(f)

In [6]:
TEST_DIR = "/home/cmchang/DLCV2018SPRING/final/dlcv_final_2_dataset/test/"

In [7]:
test_list = list()
for root, subdir, files in os.walk(TEST_DIR):
    for file in sorted(files):
        if '.jpg' in file:
            test_list.append(os.path.join(TEST_DIR, file))

In [8]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [9]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [10]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [11]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [12]:
Xtest = readImgList(test_list)

In [13]:
scope_name = "Model"

In [14]:
model = VGG16_GAP(scope_name=scope_name)

In [18]:
FLAG_save_dir = "/home/cmchang/DLCV2018SPRING/final/CL_v3-cont_dynamic_gap_L5_v3_rescale0-1_save_linear/"
FLAG_init_from = FLAG_save_dir + "para_dict.npy"

In [19]:
model.build(vgg16_npy_path=FLAG_init_from,
            shape=Xtest.shape[1:],
            classes=len(y_dict),
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=False)

build model started
npy file loaded
build model finished: 0s


In [20]:
dp = [1.0]
model.set_idp_operation(dp=dp)

DP under test: [1.]
[None, 218, 178, 3]
[None, 218, 178, 32]
AFTER [None, 218, 178, 32]
[None, 109, 89, 47]
AFTER [None, 109, 89, 47]
[None, 109, 89, 63]
AFTER [None, 109, 89, 63]
[None, 55, 45, 98]
AFTER [None, 55, 45, 98]
[None, 55, 45, 134]
AFTER [None, 55, 45, 134]
[None, 55, 45, 147]
AFTER [None, 55, 45, 147]
[None, 28, 23, 153]
AFTER [None, 28, 23, 153]
[None, 28, 23, 209]
AFTER [None, 28, 23, 209]
[None, 28, 23, 181]
AFTER [None, 28, 23, 181]
[None, 14, 12, 177]
AFTER [None, 14, 12, 177]
[None, 14, 12, 180]
AFTER [None, 14, 12, 180]
[None, 14, 12, 230]
AFTER [None, 14, 12, 230]
Set dp operations finished: 0s


In [21]:
def count_number_params(para_dict):
    n = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'conv' in k or 'fc' in k:
            n += get_params_shape(v[0].shape.as_list())
            n += get_params_shape(v[1].shape.as_list())
    return n

def get_params_shape(shape):
    n = 1
    for dim in shape:
        n = n*dim
    return n

def count_flops(para_dict, net_shape):
    input_shape = (3 ,32 ,32) # Format:(channels, rows,cols)
    total_flops_per_layer = 0
    input_count = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'fc' in k:
            continue
        elif 'conv' in k:
            conv_filter = v[0].shape.as_list()[3::-1] # (64 ,3 ,3 ,3)  # Format: (num_filters, channels, rows, cols)
            stride = 1
            padding = 1

            if conv_filter[1] == 0:
                n = conv_filter[2] * conv_filter[3] # vector_length
            else:
                n = conv_filter[1] * conv_filter[2] * conv_filter[3]  # vector_length

            flops_per_instance = n + ( n -1)    # general defination for number of flops (n: multiplications and n-1: additions)

            num_instances_per_filter = (( input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # for rows
            num_instances_per_filter *= ((input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # multiplying with cols

            flops_per_filter = num_instances_per_filter * flops_per_instance
            total_flops_per_layer += flops_per_filter * conv_filter[0]  # multiply with number of filters

            total_flops_per_layer += conv_filter[0] * input_shape[1] * input_shape[2]

            input_shape = net_shape[input_count].as_list()[3:0:-1]
            input_count +=1

    total_flops_per_layer += net_shape[-1].as_list()[1] *2360*2
    return total_flops_per_layer

def countFlopsParas(net):
    total_flops = count_flops(net.para_dict, net.net_shape)
    if total_flops / 1e9 > 1:   # for Giga Flops
        print(total_flops/ 1e9 ,'{}'.format('GFlops'))
    else:
        print(total_flops / 1e6 ,'{}'.format('MFlops'))

    total_params = count_number_params(net.para_dict)

    if total_params / 1e9 > 1:   # for Giga Flops
        print(total_params/ 1e9 ,'{}'.format('G'))
    else:
        print(total_params / 1e6 ,'{}'.format('M'))
    
    return total_flops, total_params

In [22]:
flops, params = countFlopsParas(model)
print("Flops: %3f M, Paras: %3f M" % (flops/1e6, params/1e6))
FLAG_flops_M = flops/1e6
FLAG_params_M = params/1e6

5.593132123 GFlops
4.048755 M
Flops: 5593.132123 M, Paras: 4.048755 M


In [23]:
flops, params = countFlopsParas(model)
print("Flops: %3f M, Paras: %3f M" % (flops/1e6, params/1e6))
FLAG_flops_M = flops/1e6
FLAG_params_M = params/1e6

5.593132123 GFlops
4.048755 M
Flops: 5593.132123 M, Paras: 4.048755 M


In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.global_variables())
    print("Initialized")
    output = []
    for dp_i in dp:
        for i in range(int(Xtest.shape[0]/200+1)):
            print(i, end="\r")
            st = i*200
            ed = min((i+1)*200, Xtest.shape[0])
            prob = sess.run(model.prob_dict[str(int(dp_i*100))], feed_dict={model.x: Xtest[st:ed,:], 
                                                                        model.is_train: False})
            output.append(prob)

Initialized


In [25]:
pred_prob = np.concatenate(output)
pred_class = np.argmax(pred_prob, 1)
final_id = list()
for pred in pred_class:
    final_id.append(inv_y_dict[pred])

In [26]:
doutput = pd.DataFrame({'id':np.arange(len(final_id))+1,
              'ans':final_id}, columns=['id','ans'])

In [27]:
doutput.to_csv("pred_val81_CL_v3-cont_dynamic_gap_L5_v3_rescale0-1_save_linear.csv",index=False)

In [ ]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

In [ ]:
dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

In [ ]:
valid_list = list(VALID_DIR+dvalid.img)

In [ ]:
Xvalid = readImgList(valid_list)

In [ ]:
yvalid = transformLabel(list(dvalid.id), y_dict)

In [ ]:
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [ ]:
batch_size = 200
with tf.Session() as sess:
    with tf.variable_scope(name_or_scope=scope_name) as scop:
        sess.run(tf.global_variables_initializer())
         # validation
        val_accu = 0
        for i in range(int(Xvalid.shape[0]/batch_size)):
            print(i, end='\r')
            st = i*batch_size
            ed = (i+1)*batch_size
            accu = sess.run(model.accu_dict['100'],
                                feed_dict={model.x: Xvalid[st:ed,:],
                                            model.y: Yvalid[st:ed,:],
                                            model.is_train: False})
            val_accu += accu
        val_accu = val_accu/int(Xvalid.shape[0]/batch_size)
        print("val accu : %.4f" % ( val_accu))
    

In [ ]:
# %load utils.py
from PIL import Image
import os
import errno
import sys
import pickle
import numpy as np

def gammaSparsifyVGG16(para_dict, thresh=0.5):
    last = None
    sparse_dict = {}
    N_total, N_remain = 0., 0.
    for k, v in sorted(para_dict.items()):
        if 'gamma' in k:
            # trim networks based on gamma
            gamma = v                      
            this = np.where(np.abs(gamma) > thresh)[0]
            sparse_dict[k] = gamma[this] 
            
            # get the layer name
            key = str.split(k,'_gamma')[0]
            
            # trim conv
            conv_, bias_ = para_dict[key]
            conv_ = conv_[:,:,:,this]
            if last is not None:
                conv_ = conv_[:,:,last,:]
            bias_ = bias_[this]
            sparse_dict[key] = [conv_, bias_]
            
            # get corresponding beta, bn_mean, bn_variance
            sparse_dict[key+"_beta"] = para_dict[key+"_beta"][this]
            sparse_dict[key+"_bn_mean"] = para_dict[key+"_bn_mean"][this]
            sparse_dict[key+"_bn_variance"] = para_dict[key+"_bn_variance"][this]
            
            # update
            last = this
            print('%s from %s to %s : %s ' % (k, len(gamma), len(this), len(this)/len(gamma)))
            N_total += len(gamma)
            N_remain += len(this)
    print('sparsify %s percentage' % (N_remain/N_total))
    
    W_ = para_dict['fc_2'][0][last,:] # np.concatenate([42*i+np.arange(42) for i in list(last)])
    b_ = para_dict['fc_2'][1]
    
    sparse_dict['fc_2'] = [W_, b_]
    return sparse_dict, N_remain/N_total

def dpSparsifyVGG16(para_dict, dp):
    """
    dp: usage percentage of channels in each layer
    """
    # new_dict = {}
    # first = True
    # for k,v in sorted(para_dict.items()):
    #     if 'conv1_1_' in k:
    #         new_dict[k] = v
    #     elif 'bn_mean' in k:
    #         new_dict[k] = v[:int(len(v)*dp)]
    #     elif 'bn_variance' in k:
    #         new_dict[k] = v[:int(len(v)*dp)]
    #     elif 'gamma' in k:
    #         new_dict[k] = v[:int(len(v)*dp)]
    #     elif 'beta' in k:
    #         new_dict[k] = v[:int(len(v)*dp)]
    #     elif 'fc_1' in k:
    #         O = v[0].shape[0]
    #         new_dict[k] = [v[0][:int(O*dp),:], v[1]]
    #     elif 'conv' in k:
    #         O = v[0].shape[3]
    #         if first:
    #             new_dict[k] = v[0][:,:,:,:], v[1][:] #int(O*dp)
    #             first = False
    #             last = O
    #         else:
    #             new_dict[k] = v[0][:,:,:last,:int(O*dp)], v[1][:int(O*dp)]
    #             last = int(O*dp)
    #     else:
    #         new_dict[k] = v
    #         continue
    # return new_dict
    first = True
    new_dict = {}
    last = 3
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            new_dict[k] = v[:int(len(v)*dp)]
            print("%s:%s" % (k, new_dict[k].shape))
        elif 'bn_variance' in k:
            new_dict[k] = v[:int(len(v)*dp)]
            print("%s:%s" % (k, new_dict[k].shape))
        elif 'gamma' in k:
            new_dict[k] = v[:int(len(v)*dp)]
            print("%s:%s" % (k, new_dict[k].shape))
        elif 'beta' in k:
            new_dict[k] = v[:int(len(v)*dp)]
            print("%s:%s" % (k, new_dict[k].shape))
        elif 'fc_2' in k:
            O = v[0].shape[0]
            new_dict[k] = [v[0][:int(O*dp),:], v[1]]
        elif 'conv' in k:
            O = v[0].shape[3]
            new_dict[k] = v[0][:,:,:last,:int(O*dp)], v[1][:int(O*dp)]
            # if first:
            #     first = False
            #     last = O
            # else:
            last = int(O*dp)
            print("W%s:%s" % (k, new_dict[k][0].shape))
            print("b%s:%s" % (k, new_dict[k][1].shape))
        else:
            new_dict[k] = v
    return new_dict

def count_number_params(para_dict):
    n = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'conv' in k or 'fc' in k:
            n += get_params_shape(list(v[0].shape))
            n += get_params_shape(list(v[1].shape))
    return n

def get_params_shape(shape):
    n = 1
    for dim in shape:
        n = n*dim
    return n

def count_flops(para_dict, net_shape):
    input_shape = (3 ,32 ,32) # Format:(channels, rows,cols)
    total_flops_per_layer = 0
    input_count = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'fc' in k:
            continue
        elif 'conv' in k:
            conv_filter = v[0].shape.as_list()[3::-1] # (64 ,3 ,3 ,3)  # Format: (num_filters, channels, rows, cols)
            stride = 1
            padding = 1

            if conv_filter[1] == 0:
                n = conv_filter[2] * conv_filter[3] # vector_length
            else:
                n = conv_filter[1] * conv_filter[2] * conv_filter[3]  # vector_length

            flops_per_instance = n + ( n -1)    # general defination for number of flops (n: multiplications and n-1: additions)

            num_instances_per_filter = (( input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # for rows
            num_instances_per_filter *= ((input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # multiplying with cols

            flops_per_filter = num_instances_per_filter * flops_per_instance
            total_flops_per_layer += flops_per_filter * conv_filter[0]  # multiply with number of filters

            total_flops_per_layer += conv_filter[0] * input_shape[1] * input_shape[2]

            input_shape = net_shape[input_count].as_list()[3:0:-1]
            input_count +=1

    total_flops_per_layer += net_shape[-1].as_list()[3] * 512 *2 + 512*10*2
    return total_flops_per_layer

def countFlopsParas(net):
    total_flops = count_flops(net.para_dict, net.net_shape)
    if total_flops / 1e9 > 1:   # for Giga Flops
        print(total_flops/ 1e9 ,'{}'.format('GFlops'))
    else:
        print(total_flops / 1e6 ,'{}'.format('MFlops'))

    total_params = count_number_params(net.para_dict)

    if total_params / 1e9 > 1:   # for Giga Flops
        print(total_params/ 1e9 ,'{}'.format('G'))
    else:
        print(total_params / 1e6 ,'{}'.format('M'))
    
    return total_flops, total_params

In [ ]:
dpSparsifyVGG16()

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
FLAG_save_dir = "/home/cmchang/DLCV2018SPRING/final/dynamic_gap_L5_v3_rescale0-1_save_linear/"

In [ ]:
a = np.load(FLAG_save_dir+"para_dict.npy", encoding="latin1").item()

In [ ]:
b = dpSparsifyVGG16(a, 0.5)

In [ ]:
# plt.plot(range(len(a['conv1_1_gamma'])), a['conv1_1_gamma'], label="conv1")
# plt.plot(range(len(a['conv2_1_gamma'])), a['conv2_1_gamma'], label="conv2")
# plt.plot(range(len(a['conv3_1_gamma'])), a['conv3_1_gamma'], label="conv3")
# plt.plot(range(len(a['conv4_1_gamma'])), a['conv4_1_gamma'], label="conv4")
# plt.plot(range(len(a['conv5_1_gamma'])), a['conv5_2_gamma'], label="conv5")
plt.legend()

In [ ]:
th = 8e-2

In [ ]:
b, r = gammaSparsifyVGG16(para_dict=a, thresh=th)

In [ ]:
np.save(os.path.join(FLAG_save_dir, "sparse_dict_"+str(th)+".npy"), b)